In [86]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns


In [98]:
from sklearn.impute import KNNImputer

In [88]:
df = pd.read_csv(r'C:\Users\kusha\OneDrive\Desktop\diabetes_prediction_dataset.csv')


In [90]:
df.replace({"???": None, "Unknown": None}, inplace=True)
df['HbA1c_level'] = df.groupby('diabetes')['HbA1c_level'].transform(lambda x: x.fillna(x.median()))


In [100]:
knn_imputer = KNNImputer(n_neighbors=5)
df[['bmi', 'blood_glucose_level', 'HbA1c_level']] = knn_imputer.fit_transform(df[['bmi', 'blood_glucose_level', 'HbA1c_level']])



In [94]:
ordinal_mapping = {
    'Physical Activity Level': {'Low': 0, 'Moderate': 1, 'High': 2}
}
for col, mapping in ordinal_mapping.items():
    if col in df.columns:
        df[col] = df[col].map(mapping)



In [55]:
df = pd.get_dummies(df, columns=['smoking_history', 'gender'], drop_first=True)


In [57]:
scaler = StandardScaler()
df[['blood_glucose_level', 'HbA1c_level']] = scaler.fit_transform(df[['blood_glucose_level', 'HbA1c_level']])


In [59]:
minmax_scaler = MinMaxScaler()
df[['bmi', 'age']] = minmax_scaler.fit_transform(df[['bmi', 'age']])


In [61]:
Q1 = df[['bmi', 'blood_glucose_level']].quantile(0.25)
Q3 = df[['bmi', 'blood_glucose_level']].quantile(0.75)
IQR = Q3 - Q1
df = df[~((df[['bmi', 'blood_glucose_level']] < (Q1 - 1.5 * IQR)) | (df[['bmi', 'blood_glucose_level']] > (Q3 + 1.5 * IQR))).any(axis=1)]


In [63]:
X = df.drop(columns=['diabetes'])
y = df['diabetes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [67]:
corr_matrix = X_train.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.75)]


In [69]:
X_train.drop(columns=to_drop, inplace=True)
X_test.drop(columns=to_drop, inplace=True)
print(f"Dropped highly correlated features: {to_drop}")


Dropped highly correlated features: []


In [71]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
feature_importances = pd.Series(rf.feature_importances_, index=X_train.columns).sort_values(ascending=False)
selected_features = feature_importances[feature_importances > 0.01].index.tolist()


In [72]:
X_train = X_train[selected_features]
X_test = X_test[selected_features]


In [73]:
print(f"Selected Features: {selected_features}")


Selected Features: ['HbA1c_level', 'blood_glucose_level', 'bmi', 'age', 'hypertension', 'heart_disease']


In [74]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)



In [75]:
print(f"Final Train Shape: {X_train_resampled.shape}, Test Shape: {X_test.shape}")

Final Train Shape: (137426, 6), Test Shape: (18248, 6)


In [ ]:
#checking 

In [102]:
print(X_train_resampled.isnull().sum())  
print(X_test.isnull().sum())  


HbA1c_level            0
blood_glucose_level    0
bmi                    0
age                    0
hypertension           0
heart_disease          0
dtype: int64
HbA1c_level            0
blood_glucose_level    0
bmi                    0
age                    0
hypertension           0
heart_disease          0
dtype: int64


In [104]:
print(X_train_resampled.dtypes)
print(X_test.dtypes)


HbA1c_level            float64
blood_glucose_level    float64
bmi                    float64
age                    float64
hypertension             int64
heart_disease            int64
dtype: object
HbA1c_level            float64
blood_glucose_level    float64
bmi                    float64
age                    float64
hypertension             int64
heart_disease            int64
dtype: object


In [106]:
print(X_train_resampled.mean(axis=0))  
print(X_train_resampled.std(axis=0))   


HbA1c_level            0.605573
blood_glucose_level    0.298279
bmi                    0.206255
age                    0.640798
hypertension           0.147861
heart_disease          0.091598
dtype: float64
HbA1c_level            1.206217
blood_glucose_level    0.974634
bmi                    0.053491
age                    0.271290
hypertension           0.354964
heart_disease          0.288459
dtype: float64


In [ ]:
#visualization